# TlF B state Hamiltonian
## Intro
This notebook evaluates the Hamiltonian for the B state of thallium fluoride.

In [1]:
import numpy as np
import sympy
from sympy import sqrt
import multiprocessing
import pickle

### Representing the states

A state, in general, can be written as a weighted superposition of the basis states. We work in the basis $|J, m_J, I_1, m_1, I_2, m_2\rangle$.

The operations we can define on the basis states are:

- construction: e.g. calling `BasisState(QN)` creates a basis state with quantum numbers `QN = (J, mJ, I1, m1, I2, m2)`;
- equality testing;
- inner product, returning either 0 or 1;
- superposition and scalar multiplication, returning a `State` object
- a convenience function to print out all quantum numbers

In [2]:
class BasisState:
    # constructor
    def __init__(self, J, mJ, I1, m1, I2, m2):
        self.J, self.mJ  = J, mJ
        self.I1, self.m1 = I1, m1
        self.I2, self.m2 = I2, m2

    # equality testing
    def __eq__(self, other):
        return self.J==other.J and self.mJ==other.mJ \
                    and self.I1==other.I1 and self.I2==other.I2 \
                    and self.m1==other.m1 and self.m2==other.m2

    # inner product
    def __matmul__(self, other):
        if self == other:
            return 1
        else:
            return 0

    # superposition: addition
    def __add__(self, other):
        if self == other:
            return State([ (2,self) ])
        else:
            return State([
                (1,self), (1,other)
            ])

    # superposition: subtraction
    def __sub__(self, other):
        return self + (-1)*other

    # scalar product (psi * a)
    def __mul__(self, a):
        return State([ (a, self) ])

    # scalar product (a * psi)
    def __rmul__(self, a):
        return self * a
    
    def print_quantum_numbers(self):
        print( self.J,"%+d"%self.mJ,"%+0.1f"%self.m1,"%+0.1f"%self.m2 )

A general state `State` can have any number of components, so let's represent it as an list of pairs `(amp, psi)`, where `amp` is the relative amplitude of a component, and `psi` is a basis state. The same component must not appear twice on the list.

There are three operations we can define on the states:

- construction
- superposition: concatenate component arrays and return a `State`
- scalar multiplication `a * psi` and `psi * a`, division, negation
- component-wise inner product `psi1 @ psi2`, where `psi1` is a bra, and `psi2` a ket, returning a complex number

In addition, I define an iterator method to loop through the components, and the `__getitem__()` method to access the components (which are not necessarily in any particular order!). See [Classes/Iterators](https://docs.python.org/3/tutorial/classes.html#iterators) for details.

In [3]:
class State:
    # constructor
    def __init__(self, data=[], remove_zero_amp_cpts=True):
        # check for duplicates
        for i in range(len(data)):
            amp1,cpt1 = data[i][0], data[i][1]
            for amp2,cpt2 in data[i+1:]:
                if cpt1 == cpt2:
                    raise AssertionError("duplicate components!")
        # remove components with zero amplitudes
        if remove_zero_amp_cpts:
            self.data = [(amp,cpt) for amp,cpt in data if amp!=0]
        else:
            self.data = data
        # for iteration over the State
        self.index = len(self.data)

    # superposition: addition
    # (highly inefficient and ugly but should work)
    def __add__(self, other):
        data = []
        # add components that are in self but not in other
        for amp1,cpt1 in self.data:
            only_in_self = True
            for amp2,cpt2 in other.data:
                if cpt2 == cpt1:
                    only_in_self = False
            if only_in_self:
                data.append((amp1,cpt1))
        # add components that are in other but not in self
        for amp1,cpt1 in other.data:
            only_in_other = True
            for amp2,cpt2 in self.data:
                if cpt2 == cpt1:
                    only_in_other = False
            if only_in_other:
                data.append((amp1,cpt1))
        # add components that are both in self and in other
        for amp1,cpt1 in self.data:
            for amp2,cpt2 in other.data:
                if cpt2 == cpt1:
                    data.append((amp1+amp2,cpt1))
        return State(data)
                
    # superposition: subtraction
    def __sub__(self, other):
        return self + -1*other

    # scalar product (psi * a)
    def __mul__(self, a):
        return State( [(a*amp,psi) for amp,psi in self.data] )

    # scalar product (a * psi)
    def __rmul__(self, a):
        return self * a
    
    # scalar division (psi / a)
    def __truediv__(self, a):
        return self * (1/a)
    
    # negation
    def __neg__(self):
        return -1 * self
    
    # inner product
    def __matmul__(self, other):
        result = 0
        for amp1,psi1 in self.data:
            for amp2,psi2 in other.data:
                result += amp1.conjugate()*amp2 * (psi1@psi2)
        return result

    # iterator methods
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index -= 1
        return self.data[self.index]
    
    # direct access to a component
    def __getitem__(self, i):
        return self.data[i]

### Operators in Python

Define QM operators as Python functions that take `BasisState` objects, and return `State` objects. Since we are interested in finding matrix elements, we only need the action of operators on the basis states (but it'd be easy to generalize using a `for` loop).

The easiest operators to define are the diagonal ones $J^2, J_z, I_{1z}, I_{2z}$, which just multiply the state by their eigenvalue:

In [4]:
def J2(psi):
    return State([(psi.J*(psi.J+1),psi)])

def Jz(psi):
    return State([(psi.mJ,psi)])

def I1z(psi):
    return State([(psi.m1,psi)])

def I2z(psi):
    return State([(psi.m2,psi)])

The other angular momentum operators we can obtain through the ladder operators

$$ J_\pm=J_x\pm iJ_y. $$

These are defined through their action on the basis states as (Sakurai eqns 3.5.39-40)

$$ J_\pm|J,m\rangle=\sqrt{(j\mp m)(j\pm m+1)}|jm\pm1\rangle. $$

Similarly, $I_{1\pm},I_{2\pm}$ act on the $|I_1,m_1\rangle$ and $|I_2,m_2\rangle$ subspaces in the same way.

In [5]:
def Jp(psi):
    amp = sqrt((psi.J-psi.mJ)*(psi.J+psi.mJ+1))
    ket = BasisState(psi.J, psi.mJ+1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp,ket)])

def Jm(psi):
    amp = sqrt((psi.J+psi.mJ)*(psi.J-psi.mJ+1))
    ket = BasisState(psi.J, psi.mJ-1, psi.I1, psi.m1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I1p(psi):
    amp = sqrt((psi.I1-psi.m1)*(psi.I1+psi.m1+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1+1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I1m(psi):
    amp = sqrt((psi.I1+psi.m1)*(psi.I1-psi.m1+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1-1, psi.I2, psi.m2)
    return State([(amp,ket)])

def I2p(psi):
    amp = sqrt((psi.I2-psi.m2)*(psi.I2+psi.m2+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2+1)
    return State([(amp,ket)])

def I2m(psi):
    amp = sqrt((psi.I2+psi.m2)*(psi.I2-psi.m2+1))
    ket = BasisState(psi.J, psi.mJ, psi.I1, psi.m1, psi.I2, psi.m2-1)
    return State([(amp,ket)])

In terms of the above-defined ladder operators, we can write

$$J_x=\frac{1}{2}(J_++J_-);\quad
J_y=\frac{1}{2i}(J_+-J_-),$$

and similarly for $I_{1x}, I_{1y}$ and $I_{2x}, I_{2y}$.

In [6]:
def Jx(psi):
    return half*( Jp(psi) + Jm(psi) )

def Jy(psi):
    return -half*1j*( Jp(psi) - Jm(psi) )

def I1x(psi):
    return half*( I1p(psi) + I1m(psi) )

def I1y(psi):
    return -half*1j*( I1p(psi) - I1m(psi) )

def I2x(psi):
    return half*( I2p(psi) + I2m(psi) )

def I2y(psi):
    return -half*1j*( I2p(psi) - I2m(psi) )

### Composition of operators

All operators defined above can only accept `BasisStates` as their inputs, and they all return `States` as output. To allow composition of operators,

$$\hat A\hat B|\psi\rangle=\hat A(\hat B(|\psi\rangle)),$$

define the following function.

In [7]:
def com(A, B, psi):
    ABpsi = State()
    # operate with A on all components in B|psi>
    for amp,cpt in B(psi):
        ABpsi += amp * A(cpt)
    return ABpsi

### Rotational term

The simplest term in the Hamiltonian simply gives the rotational levels:

$$H_\text{rot}=B_\text{rot}\vec J^2.$$

In [8]:
def Hrot(psi):
    return Brot * J2(psi)

### Electron magnetic hyperfine operator
Since the B state is a triplet pi state, it has electron magnetic hyperfine structure. The coupling is described by the Hamiltonian $ H_{\mathrm{mhf}} = a\, \mathbf{I} \cdot \mathbf{L} + b\, \mathbf{I} \cdot \mathbf{S} +c \,I_z\, S_z  $ which reduces to $ H_{\mathrm{mhf}}^{eff} =  [a \,L_z + (b+c)\, S_z]\,I_z = h_\Omega \, I_z $ since the raising and lowering operators for L and S (electron orbital and spin angular momentum) only couple different electronic states which are very far in energy and thus the effect of the off-diagonal elements is strongly suppressed. The z-direction in $I_z$ is along the internuclear axis of the molecule and therefore when evaluating the matrix elements one needs to rotate the spin operator to the lab fixed frame. This results in non-zero matrix elements between states with different J.

In the uncoupled basis these matrix elements are given by

\begin{equation*}
\mathrm{<J, m_J, \Omega, I_1, m_1, I_2, m_2 |\, I_z(Tl)\, | J', m'_J, \Omega', I'_1, m'_1, I'_2, m'_2>} \\
= - 
\begin{pmatrix}
J & 1 & J' \\
-\Omega & 0 & \Omega'
\end{pmatrix}
\lbrack (2J+1)(2J'+1)I_1(I_1+1)(2I_1+1) \rbrack^{\frac{1}{2}} \delta_{I_2,I'_2} \delta_{m_2,m'_2} \\
\sum_{p=\,-1}^{+1} (-1)^{p-m_J+I_1-m_1}
\begin{pmatrix}
J & 1 & J' \\
-m_J & -p & m'_J
\end{pmatrix}
\begin{pmatrix}
I_1 & 1 & I'_1 \\
-m_1 & p & m'_1
\end{pmatrix}
\end{equation*}

for Tl and 

\begin{equation*}
\mathrm{<J, m_J, \Omega, I_1, m_1, I_2, m_2 |\, I_z(Tl)\, | J', m'_J, \Omega', I'_1, m'_1, I'_2, m'_2>} \\
= - 
\begin{pmatrix}
J & 1 & J' \\
-\Omega & 0 & \Omega'
\end{pmatrix}
\lbrack (2J+1)(2J'+1)I_2(I_2+1)(2I_2+1) \rbrack^{\frac{1}{2}} \delta_{I_1,I'_1} \delta_{m_1,m'_1} \\
\sum_{p=\,-1}^{+1} (-1)^{p-m_J+I_2-m_2}
\begin{pmatrix}
J & 1 & J' \\
-m_J & -p & m'_J
\end{pmatrix}
\begin{pmatrix}
I_2 & 1 & I'_2 \\
-m_2 & p & m'_2
\end{pmatrix}
\end{equation*}

for F

In [9]:
#Import Wigner 3j symbol
from sympy.physics.wigner import wigner_3j

def H_mhf_Tl(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    omega = 1
    
    #I1, I2 and m2 must be the same for non-zero matrix element
    I2prime = I2
    m2prime = m2
    I1prime = I1
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over the possible values of quantum numbers for which the matrix element can be non-zero
    #Need Jprime = J+1 ... |J-1|
    for Jprime in np.arange(np.abs(J-1), J+2):
        #Evaluate the part of the matrix element that is common for all p
        common_coefficient = -h1_Tl*wigner_3j(J, 1, Jprime, -omega, 0, omega)*sqrt((2*J+1)*(2*Jprime+1)*I1*(I1+1)*(2*I1+1))
        
        #Loop over the spherical tensor components of I1:
        for p in np.arange(-1,2):
            #To have non-zero matrix element need mJ-p = mJprime
            mJprime = mJ + p
            
            #Also need m2 - p = m2prime
            m1prime = m1 - p
            
            #Check that mJprime and m2prime are physical
            if np.abs(mJprime) <= Jprime and np.abs(m1prime) <= I1prime:
                #Calculate rest of matrix element
                p_factor = ((-1)**(p-mJ+I1-m1)*wigner_3j(J, 1, Jprime, -mJ, -p, mJprime)
                               *wigner_3j(I1, 1, I1prime, -m1, p, m1prime))
                               
                amp = common_coefficient*p_factor
                basis_state = BasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime)
                if amp != 0:
                    data.append((amp, basis_state))
    
    return State(data)
            

In [10]:
def H_mhf_F(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    mJ = psi.mJ
    I1 = psi.I1
    m1 = psi.m1
    I2 = psi.I2
    m2 = psi.m2
    omega = 1
    
    #I1, I2 and m1 must be the same for non-zero matrix element
    I1prime = I1
    m1prime = m1
    I2prime = I2
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over the possible values of quantum numbers for which the matrix element can be non-zero
    #Need Jprime = J+1 ... |J-1|
    for Jprime in np.arange(np.abs(J-1), J+2):
        #Evaluate the part of the matrix element that is common for all p
        common_coefficient = -h1_F*wigner_3j(J, 1, Jprime, -omega, 0, omega)*sqrt((2*J+1)*(2*Jprime+1)*I2*(I2+1)*(2*I2+1))
        
        #Loop over the spherical tensor components of I2:
        for p in np.arange(-1,2):
            #To have non-zero matrix element need mJ-p = mJprime
            mJprime = mJ + p
            
            #Also need m2 - p = m2prime
            m2prime = m2 - p
            
            #Check that mJprime and m2prime are physical
            if np.abs(mJprime) <= Jprime and np.abs(m2prime) <= I2prime:
                #Calculate rest of matrix element
                p_factor = ((-1)**(p-mJ+I2-m2)*wigner_3j(J, 1, Jprime, -mJ, -p, mJprime)
                               *wigner_3j(I2, 1, I2prime, -m2, p, m2prime))
                               
                amp = common_coefficient*p_factor
                basis_state = BasisState(Jprime, mJprime, I1prime, m1prime, I2prime, m2prime)
                if amp != 0:
                    data.append((amp, basis_state))
        
        
    return State(data)

In [11]:
def Hff(psi):
    return Hrot(psi) + H_mhf_Tl(psi) + H_mhf_F(psi)

### Finding the matrix elements

With all the operators defined, we can evaluate the matrix elements for a given range of quantum numbers. We shall need to generate a non-integer range list (e.g., from -3/2 to +3/2):

In [12]:
def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(sympy.Number(x))
        x += dx
    return range_list

Define constants for TlF:

In [17]:
half = sympy.Rational(1,2)
#half = 0.5

Jmin = sympy.Integer(1)
Jmax = sympy.Integer(3) # max J value in Hamiltonian
#Jmax = 6
I_Tl = half             # I1 in Ramsey's notation
I_F  = half             # I2 in Ramsey's notation

Brot = sympy.symbols('Brot')
h1_Tl, h1_F = sympy.symbols('h1_Tl h1_F')
c1, c2, c3, c4 = sympy.symbols('c1 c2 c3 c4')
D_TlF = sympy.symbols('D_TlF')
mu_J, mu_Tl, mu_F = sympy.symbols('mu_J mu_Tl mu_F')

 Write down the basis as a list of `BasisState` components:

In [18]:
QN = [BasisState(J,mJ,I_Tl,m1,I_F,m2)
      for J  in ni_range(Jmin, Jmax+1)
      for mJ in ni_range(-J,J+1)
      for m1 in ni_range(-I_Tl,I_Tl+1)
      for m2 in ni_range(-I_F,I_F+1)
     ]

#Write down a basis where the mJ, m1 and m2 values all sum to zero so mF = 0
# QN = []

# for J in np.arange(Jmin, Jmax+1):
#     for m1 in np.arange(-I_Tl,I_Tl+1):
#         for m2 in np.arange(-I_F,I_F+1):
#             mJ = -m1-m2
            
#             if np.abs(mJ) <= J:
#                 QN.append(BasisState(J,mJ,I_Tl,m1,I_F,m2))

The field-free and Stark/Zeeman components of the Hamiltonian then have the matrix elements (evaluate using `multiprocessing` to speed things up)

In [19]:
%%time
from tqdm import tqdm

def HMatElems(H, QN=QN):
    result = sympy.zeros(len(QN),len(QN))
    for i,a in tqdm(enumerate(QN)):
        for j,b in enumerate(QN):
            result[i,j] = (1*a)@H(b)
            
    return result

H_ops = [Hff]
Hff_m = HMatElems(Hff)

60it [00:26,  2.30it/s]

Wall time: 26.1 s


Store the result of the calculation as text files and Python `pickle`s:

In [20]:
with open("B_hamiltonians_symbolic.py", 'wb') as f:
    pickle.dump(
        {
            "Hff" : Hff_m
        },
        f
    )

In [ ]:
with open("B_hamiltonians_symbolic.txt", 'w') as f:
    f.write(
        str(
            {
                "Hff" : Hff_m
            }
        )
    )